In [ ]:
import scanpy as sc

In [ ]:
dir_path = "/home/krushna/Documents/Data_integration/SCRNA_Datasets/All_h5ad/"
def load_data(dataset,batch):
    adata =sc.read_h5ad(dir_path+dataset+'.h5ad')
    sc.pp.filter_genes(adata, min_counts=3)
    adata.layers["counts"] = adata.X.copy()
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    adata.raw = adata
    sc.pp.highly_variable_genes(
            adata,
            flavor="seurat",
            n_top_genes=2000,
            layer="counts",
            batch_key=batch,
            subset=True
    )
    return adata
    
batch_key_dic = {'Immune_Human' : 'batch',
                 'Immune_human_mouse' : 'batch',
                 'Lung' : 'batch',
                 'Mouse_brain' : 'batch',
                 'Pancreas' : 'tech',
                 'Simulation1' : 'Batch',
                 'Simulation2' : 'Batch',
                 'Pancreas_scDGN' : 'batch',
                 'Mouse_Brain_scDGN' : 'batch',
                 'tabular_muris' : "batch",
                 'Human_Mouse' : 'batch',
                 'Human_Retina': "Batch"
                 
                }
cell_type_key_dic = {'Immune_Human' : 'final_annotation',
                 'Immune_human_mouse' : 'final_annotation',
                 'Lung' : 'cell_type',
                 'Mouse_brain' : 'cell_type',
                 'Pancreas' : 'celltype',
                 'Simulation1' : 'Group',
                 'Simulation2' : 'Group',
                 'Pancreas_scDGN' : 'cell_type',
                 'Mouse_Brain_scDGN' : "tissue" ,
                 "tabular_muris" : "cell_ontology_class",
                 'Human_Mouse' : "celltype",
                 "Human_Retina":"Subcluster"
                    }  

In [ ]:
dataset = 'Immune_Human_removed_top_cells'
batch = 'batch'
cell_type = 'final_annotation'
adata = load_data(dataset,batch)

In [ ]:
# Load data
import pandas as pd
import harmonypy as hm


sc.tl.pca(adata)
meta_data = adata.obs
vars_use = [batch]

data_mat = adata.obsm['X_pca'][:,:20]
# meta_data
#
#                  cell_id dataset  nGene  percent_mito cell_type
# 0    half_TGAAATTGGTCTAG    half   3664      0.017722    jurkat
# 1    half_GCGATATGCTGATG    half   3858      0.029228      t293
# 299  t293_ACGCTGCTTCTTAC    t293   3513      0.021240      t293



# data_mat[:5,:5]
#
# array([[ 0.0071695 , -0.00552724, -0.0036281 , -0.00798025,  0.00028931],
#        [-0.011333  ,  0.00022233, -0.00073589, -0.00192452,  0.0032624 ],
#        [ 0.0091214 , -0.00940727, -0.00106816, -0.0042749 , -0.00029096],
#        [ 0.00866286, -0.00514987, -0.0008989 , -0.00821785, -0.00126997],
#        [-0.00953977,  0.00222714, -0.00374373, -0.00028554,  0.00063737]])

# meta_data.shape # 300 cells, 5 variables
# (300, 5)
#
# data_mat.shape  # 300 cells, 20 PCs
# (300, 20)

# Run Harmony

# res.to_csv("data/adj.tsv.gz", sep = "\t", index = False)


In [ ]:
ho = hm.run_harmony(data_mat, meta_data, vars_use)

# Write the adjusted PCs to a new file.
res = pd.DataFrame(ho.Z_corr)


In [ ]:
adata.obsm['X_harmony']  = res.T.to_numpy()
adata.write('harmony-Immune_Human-removed_top_cells.h5ad')

In [ ]:
adata = sc.read_h5ad('harmony-Immune_Human-removed_top_cells.h5ad')

In [ ]:
import scIB 
#Trajectory is asking precomputed sudo time point
results,ilisi_all,clisi_all,kbet_all  =      scIB.metrics.metrics(
        adata,
        adata,
        batch_key = batch,
        label_key = cell_type,
        hvg_score_=False,
        cluster_key='cluster',
        cluster_nmi=None,
        ari_=True,
        nmi_=True,
        nmi_method='arithmetic',
        nmi_dir=None,
        silhouette_=True,
        embed= 'X_harmony',
        si_metric='euclidean',
        pcr_=True,
        cell_cycle_=False,
        organism='mouse',
        isolated_labels_=True,  # backwards compatibility
        isolated_labels_f1_=True,
        isolated_labels_asw_=True,
        n_isolated=None,
        graph_conn_=True,
        kBET_=True,
        kBET_sub=0.5,
        lisi_graph_=True,
        lisi_raw=True,
        trajectory_=False,
        type_=None,
        verbose=False,
)

In [ ]:
results

In [ ]:
import numpy as np
np.savetxt(dataset+"_ilisi.csv", ilisi_all, delimiter=",")
np.savetxt(dataset+"_clisi.csv", clisi_all, delimiter=",")
np.savetxt(dataset+"_kbet_all.csv",np.concatenate([np.array(val).reshape(1,-1) for val in kbet_all],axis = 0), delimiter=',')

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi


latent_matrix = adata.obsm['X_harmony'] 
labels = np.array(adata.obs[cell_type] )
K = np.size(np.unique(labels))
kmeans = KMeans(n_clusters=K, random_state=0).fit(latent_matrix)
y_pred = kmeans.labels_

print('Computing NMI ...')
NMI = nmi(labels.flatten(), y_pred.flatten())
print('NMI = {}'.format(NMI))

In [ ]:
sc.pp.neighbors(adata, use_rep='X_harmony')  # use_rep = 'final_embeddings'
sc.tl.umap(adata)
sc.pl.umap(adata, color=cell_type, frameon=False)
sc.pl.umap(adata, color=batch, frameon=False)
sc.pl.umap(adata, color='cluster', frameon=False)

In [ ]:
adata.obs['temp'] =  y_pred
adata.obs['temp'] = adata.obs['temp'].astype(str)
sc.pl.umap(adata, color='temp', frameon=False)